In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

Using TensorFlow backend.


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 64373286793e3c8b2b4e3219cbf3544b so we will re-download the data.
108650496/553467096 [====>.........................] - ETA: 7:29

ConnectionResetError: [Errno 104] Connection reset by peer

In [ ]:
new_attribute=None

import p5_util
import P7_DataBreed
if new_attribute is None :
    is_saved = True
    filename='./data/oP7_DataBreed.dump'
    oP7_DataBreed_save = P7_DataBreed.P7_DataBreed()
    try: 
        oP7_DataBreed
    except NameError:
        print('*** INFO : oP7_DataBreed is not defined; loading...')
        oP7_DataBreed = p5_util.object_load(filename)
        is_saved = False
    oP7_DataBreed_save.copy(oP7_DataBreed)
    oP7_DataBreed = P7_DataBreed.P7_DataBreed()
    oP7_DataBreed.copy(oP7_DataBreed_save)
    del(oP7_DataBreed_save)
    oP7_DataBreed.show()
    is_saved=True
    if is_saved is True:
        print('*** INFO : oP7_DataBreed is saved')
        p5_util.object_dump(oP7_DataBreed,filename)


In [ ]:
import P7_DataBreed
oP7_DataBreed.breed_show()

In [ ]:
import P7_DataBreed

oP7_DataBreed.show_image_name('boxer')

In [ ]:
#help(oP7_DataBreed._build_pathname)
dirbreed = 'n02108089-boxer'
image_name = 'n02108089_8969.jpg'
image_pathname = oP7_DataBreed._build_pathname(dirbreed,image_name)

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img(image_pathname, target_size=(224, 224))  # Charger l'image
img = img_to_array(img)  # Convertir en tableau numpy
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16